In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
def calc_MAE(x_col,y_col):
    nan_mask = ~np.isnan(x_col) & ~np.isnan(y_col)
    zero_mask = (x_col != 0) & (y_col != 0)
    final_mask = nan_mask & zero_mask
    if np.all(~nan_mask):
        return -10000000 
    x_col_masked = x_col[final_mask]
    y_col_masked = y_col[final_mask]
    mse = mean_absolute_error(x_col_masked, y_col_masked)
    return mse

def calc_MAPE(x_col, y_col):
    nan_mask = ~np.isnan(x_col) & ~np.isnan(y_col)
    zero_mask = (x_col != 0) & (y_col != 0)
    final_mask = nan_mask & zero_mask
    if np.all(~nan_mask):
        return -10000000 
    x_col_masked = x_col[final_mask]
    y_col_masked = y_col[final_mask]
    mape = np.mean(np.abs((x_col_masked - y_col_masked) / y_col_masked)) * 100
    return mape

merged_all = pd.read_csv('../make_maps/input/pred_corr.csv')
merged_all = pd.read_csv('../make_maps/input/Prediction_corrected.csv')
merged_all = merged_all.rename(columns={'predicted_P2O5_avg_app_cor': 'predicted_P2O5_avg_app', 
                                        'predicted_K2O_avg_app_cor': 'predicted_K2O_avg_app',
                                        'predicted_N_avg_app_cor': 'predicted_N_avg_app'})
merged_all = merged_all[['FAOStat_area_code', 'Year', 'Crop_Code', 'predicted_P2O5_avg_app',
       'predicted_K2O_avg_app', 'predicted_N_avg_app']]
df_national = pd.read_csv('FUBC_National_Reports_v2 (2).csv')
df_national = df_national[['FAOStat_area_code','Year','Crop_Code','N_avg_app','K2O_avg_app','P2O5_avg_app']]
merged_all = merged_all[merged_all['Year']>=1960]
df_national = df_national[df_national['Year']>=1960]
merged_all = merged_all[merged_all['Year']<=2020]
df_national = df_national[df_national['Year']<=2020]
df_merged = pd.merge(df_national, merged_all, left_on=['Year','FAOStat_area_code','Crop_Code'], 
                      right_on=['Year','FAOStat_area_code','Crop_Code'])
df_national = pd.read_csv('FUBC_National_Reports_v2 (2).csv')
df_Totals = df_national[['FAOStat_area_code', 'Year', 'Crop_Code','NPK_avg_app']]
df_merged_Pak = pd.merge(df_Totals, merged_all, left_on=['Year','FAOStat_area_code','Crop_Code'], 
                      right_on=['Year','FAOStat_area_code','Crop_Code'])
df_merged_Pak = df_merged_Pak[df_merged_Pak['FAOStat_area_code'] == 165]
df_merged_Pak['Total_avg_app'] = df_merged_Pak['predicted_P2O5_avg_app'] + df_merged_Pak['predicted_N_avg_app']+ df_merged_Pak['predicted_K2O_avg_app']

In [ ]:
df_c_codes = pd.read_excel('Countries_R_Codes.xlsx')
df_c_codes.drop(columns=['ISO3'],inplace=True)
mae_per_code = pd.merge(df_merged, df_c_codes, left_on=['FAOStat_area_code'], 
                      right_on=['FAOStat_area_code'])
concatenated_columns = mae_per_code['Area_R'] + mae_per_code['Crop_Code']
occurrences = concatenated_columns.value_counts()

# Comparisons

### Compare with national databases

In [ ]:
mae_per_code = pd.DataFrame()
for nutrient in ['N_avg_app', 'P2O5_avg_app', 'K2O_avg_app']:
    mae_per_nutrient = df_merged.groupby(['FAOStat_area_code', 'Crop_Code']).apply(lambda x: calc_MAE(x[nutrient],
                                                                                                      x[f'predicted_{nutrient}']))
    mae_per_nutrient = mae_per_nutrient.reset_index()
    mae_per_nutrient.columns = ['FAOStat_area_code', 'Crop_Code', f'MAE_{nutrient}']
    if mae_per_code.empty:
        mae_per_code = mae_per_nutrient
    else:
        mae_per_code = pd.merge(mae_per_code, mae_per_nutrient, on=['FAOStat_area_code', 'Crop_Code'])

In [ ]:
mae_per_code = pd.DataFrame()
for nutrient in ['N_avg_app', 'P2O5_avg_app', 'K2O_avg_app']:
    mae_per_nutrient = df_merged.groupby(['FAOStat_area_code', 'Crop_Code']).apply(lambda x: calc_MAE(x[nutrient],
                                                                                                      x[f'predicted_{nutrient}']))
    mae_per_nutrient = mae_per_nutrient.reset_index()
    mae_per_nutrient.columns = ['FAOStat_area_code', 'Crop_Code', f'MAE_{nutrient}']
    if mae_per_code.empty:
        mae_per_code = mae_per_nutrient
    else:
        mae_per_code = pd.merge(mae_per_code, mae_per_nutrient, on=['FAOStat_area_code', 'Crop_Code'])
    mape_per_nutrient = df_merged.groupby(['FAOStat_area_code', 'Crop_Code']).apply(lambda x: calc_MAPE(x[nutrient],
                                                                                                        x[f'predicted_{nutrient}']))
    mape_per_nutrient = mape_per_nutrient.reset_index()
    mape_per_nutrient.columns = ['FAOStat_area_code', 'Crop_Code', f'MAPE_{nutrient}']
    mae_per_code = pd.merge(mae_per_code, mape_per_nutrient, on=['FAOStat_area_code', 'Crop_Code'])        
mae_per_code.replace(-10000000, pd.NA, inplace=True)
mae_per_code['MAE_NPK_avg_app'] = pd.NA
mae_per_code['MAPE_NPK_avg_app'] = pd.NA
for crop_code in df_merged_Pak['Crop_Code'].unique():
    subset = df_merged_Pak[df_merged_Pak['Crop_Code'] == crop_code]
    mae_per_code.loc[(mae_per_code['FAOStat_area_code'] == 165) & (mae_per_code['Crop_Code'] == crop_code),'MAE_NPK_avg_app'] = calc_MAE(subset['NPK_avg_app'],
                                                                                            subset['Total_avg_app'])
    mae_per_code.loc[(mae_per_code['FAOStat_area_code'] == 165) & (mae_per_code['Crop_Code'] == crop_code),'MAPE_NPK_avg_app'] = calc_MAPE(subset['NPK_avg_app'],
                                                                                            subset['Total_avg_app'])
df_c_codes = pd.read_excel('Countries_R_Codes.xlsx')
df_c_codes.drop(columns=['ISO3'],inplace=True)
mae_per_code = pd.merge(mae_per_code, df_c_codes, left_on=['FAOStat_area_code'], 
                      right_on=['FAOStat_area_code'])
mae_per_code.to_excel('final_corr_v1.xlsx', index=False)

# Making of plots for the output

## Making of comparative plot

In [ ]:
crop_names = {
    "5": "Sugar Crops",
    "6": "Fibre Crops",
    "1_1": "Wheat",
    "1_2": "Maize",
    "1_3": "Rice",
    "1_4": "Other Cereals",
    "2_1": "Soybean",
    "3_1": "Vegetables",
    "4": "Roots and Tubers",
    "2_3": "Other Oilseeds"
}
country_names = {
    229: "DEFRA - UK",
    231: "USDA"
}
fertilizer_list = ["P2O5","N","K2O"]
fertilizer_colors = {
    'N': '#000000',
    'P2O5': '#909090',
    'K2O': '#D8D8D8'
}
line_styles = {
    'national': '-',
    'predicted': '--'
}

In [ ]:
# Run all plots 
for country, country_name in country_names.items():
   # if country == 231:
   #     continue
    # Get unique crop codes
    subset = df_national[df_national['FAOStat_area_code'] == country]
    subset['Year'] = subset['Year'].astype(int)
    subset2 = merged_all[merged_all['FAOStat_area_code'] == country]
    subset2['Year'] = subset2['Year'].astype(int)
    set1 = set(subset['Crop_Code'].unique())
    set2 = set(subset2['Crop_Code'].unique())
    intersection = set1.intersection(set2)
    crop_codes = list(intersection)
    crop_codes = np.array(crop_codes)

    # Create subplots
    num_plots = len(crop_codes)
    num_rows = (num_plots + 1) // 2  # Ensure we have enough rows for the plots
    fig, axs = plt.subplots(num_rows, 2, figsize=(20, 10*num_rows), dpi=1200)
    axs = axs.flatten()
    # Iterate over each crop code
    for i, crop_code in enumerate(crop_codes):
        crop_name = crop_names[crop_code]
        ax = axs[i] if len(crop_codes) > 1 else axs  # Select the appropriate axis

        # Subset data for the specific crop
        subset_plots_us = merged_all[(merged_all['Crop_Code'] == crop_code) & (merged_all['FAOStat_area_code'] == country)]
        subset_plots_us = subset_plots_us.sort_values(by='Year', ascending=True) 
        subset_plots_us = subset_plots_us.reset_index(drop=True)
        if country == 229:
            subset_plots_us = subset_plots_us[subset_plots_us['Year'] >= 2000]
        if country == 231:
            subset_plots_us = subset_plots_us[subset_plots_us['Year'] >= 1960]

        df_national_sub = df_national[(df_national['Crop_Code'] == crop_code) & (df_national['FAOStat_area_code'] == country)]
        df_national_sub = df_national_sub.sort_values(by='Year', ascending=True) 
        df_national_sub = df_national_sub.reset_index(drop=True)
        if country == 229:
            df_national_sub = df_national_sub[df_national_sub['Year'] >= 2000]
            df_national_sub['Year'] = df_national_sub['Year'].astype(int)
        if country == 231:
            df_national_sub = df_national_sub[df_national_sub['Year'] >= 1960]
        df_national_sub = df_national_sub[df_national_sub['Year'] <= 2019]
        subset_plots_us = subset_plots_us[subset_plots_us['Year'] <= 2019]
        subset_plots_us = subset_plots_us.reset_index(drop=True)
        df_national_sub = df_national_sub.reset_index(drop=True)

        # Plot each fertilizer
        for j, fertilizer in enumerate(fertilizer_list):
            print(fertilizer,crop_code)
            fertilizer_pred = 'predicted_{}_avg_app'.format(fertilizer)
            fertilizer_app = '{}_avg_app'.format(fertilizer)
            color = fertilizer_colors.get(fertilizer, 'blue')  # Default color if not found in the dictionary
            linestyle = line_styles.get('predicted', '-')  # Default linestyle for predicted values
            subset_plots_us['Year'] = subset_plots_us['Year'].astype(int)
            ax.plot(subset_plots_us['Year'], subset_plots_us[fertilizer_pred], 
                    label='Our estimation - {}'.format(fertilizer), 
                    zorder=2, linestyle=linestyle, color=color)
            linestyle = line_styles.get('national', '--')  # Default linestyle for national database values
            ax.plot(df_national_sub['Year'], df_national_sub[fertilizer_app], 
                        label='{} - {}'.format(country_name, fertilizer), 
                        zorder=2, linestyle=linestyle, color=color)
                
        ax.xaxis.set_major_formatter(plt.FormatStrFormatter('%d'))        
        ax.grid(which="major", axis='x', color='#DAD8D7', alpha=0.5, zorder=1)
        ax.grid(which="major", axis='y', color='#DAD8D7', alpha=0.5, zorder=1)
        ax.yaxis.set_label_position("left")
        ax.yaxis.set_tick_params(pad=2, labeltop=False, labelbottom=True, bottom=False, labelsize=18)
        ax.xaxis.set_tick_params(pad=2, labeltop=False, labelbottom=True, bottom=False, labelsize=18)
        ax.spines[['top','right','bottom']].set_visible(False)
        ax.spines['left'].set_linewidth(1.1)
        ax.set_title("{}".format(crop_name), fontsize=24, weight='bold', alpha=.8)
        
    # Make picture nice
    fig.text(0.04, 0.5, 'Fertilizer application rate [kg ha⁻¹ year⁻¹]', 
             fontsize=22, ha='center', va='center', rotation='vertical')
    fig.text(0.5, 0.03, 'Year', fontsize=22, ha='center', va='bottom', rotation='horizontal')
    
    # Create custom legend for national database and prediction difference
    handles_difference = [plt.Line2D([0], [0], color='k', linestyle=line_styles.get('predicted', '-')),
                          plt.Line2D([0], [0], color='k', linestyle=line_styles.get('national', '--'))]
    labels_difference = ['Prediction', 'National database']
    labels_nutrients = ['N', 'P₂O₅', 'K₂O']#labels_nutrients = ['\n'.join([label, nutrient]) for label, nutrient in zip(labels_nutrients, fertilizer_list)]
    handles_nutrients = [plt.Line2D([0], [0], marker='o', color='w', markersize=18, markerfacecolor=fertilizer_colors.get(fertilizer)) for fertilizer in fertilizer_list]

    # Drop the title of the legend and add all nutrients under each other
    labels_nutrients = ['\n'.join(label.split('_')) for label in labels_nutrients]
    fig.legend(handles_nutrients[0:2] + [handles_nutrients[2]] + handles_difference, labels_nutrients[0:2]+ [labels_nutrients[2]] + labels_difference , title="", loc='lower left', fontsize=22, ncol=2, frameon=False)

    # Adjust the position of the legend
    fig.subplots_adjust(bottom=0.5)
    
    if country_name == "USDA":
        fig.suptitle("United States - USDA", fontsize=24, weight='bold')
    else:
        fig.suptitle("United Kingdom - DEFRA", fontsize=24, weight='bold')
    plt.subplots_adjust(left=None, bottom=0.1, right=None, top=0.95, wspace=None, hspace=0.3)
    fig.patch.set_facecolor('white')
    # Save image
    plt.savefig('Final_ END_PLOT_{}.png'.format(country), bbox_inches='tight',dpi=400)
    plt.savefig('Final_ END_PLOT_{}.pdf'.format(country), bbox_inches='tight',dpi=400)


# Extra

In [ ]:
df_start = pd.read_csv('../data/FUBC_full_dataset_v2.csv')
df_start = df_start[['FAOStat_area_code', 'Year', 'Crop_Code', 'P2O5_avg_app',
       'K2O_avg_app', 'N_avg_app']]
df_start = df_start.rename(columns={
    'P2O5_avg_app': 'P2O5_avg_app_F',
    'K2O_avg_app': 'K2O_avg_app_F',
    'N_avg_app': 'N_avg_app_F'
})
df_national = pd.read_csv('FUBC_National_Reports_v2 (2).csv')
df_national = df_national[['FAOStat_area_code','Year','Crop_Code','N_avg_app','K2O_avg_app','P2O5_avg_app']]
df_start = df_start[df_start['Year']>=1970]
df_national = df_national[df_national['Year']>=1970]
df_start = df_start[df_start['Year']<=2020]
df_national = df_national[df_national['Year']<=2020]
df_merged = pd.merge(df_national, df_start, left_on=['Year','FAOStat_area_code','Crop_Code'], 
                      right_on=['Year','FAOStat_area_code','Crop_Code'])
df_national = pd.read_csv('FUBC_National_Reports_v2 (2).csv')
df_Totals = df_national[['FAOStat_area_code', 'Year', 'Crop_Code','NPK_avg_app']]
df_merged_Pak = pd.merge(df_Totals, df_start, left_on=['Year','FAOStat_area_code','Crop_Code'], 
                      right_on=['Year','FAOStat_area_code','Crop_Code'])
df_merged_Pak = df_merged_Pak[df_merged_Pak['FAOStat_area_code'] == 165]
df_merged_Pak['Total_avg_app'] = df_merged_Pak['P2O5_avg_app_F'] + df_merged_Pak['K2O_avg_app_F']+ df_merged_Pak['N_avg_app_F']

mae_per_code = pd.DataFrame()
for nutrient in ['N_avg_app', 'P2O5_avg_app', 'K2O_avg_app']:
    mae_per_nutrient = df_merged.groupby(['FAOStat_area_code', 'Crop_Code']).apply(lambda x: calc_MAE(x[nutrient],
                                                                                                      x[f'{nutrient}_F']))
    mae_per_nutrient = mae_per_nutrient.reset_index()
    mae_per_nutrient.columns = ['FAOStat_area_code', 'Crop_Code', f'MAE_{nutrient}']
    if mae_per_code.empty:
        mae_per_code = mae_per_nutrient
    else:
        mae_per_code = pd.merge(mae_per_code, mae_per_nutrient, on=['FAOStat_area_code', 'Crop_Code'])
    mape_per_nutrient = df_merged.groupby(['FAOStat_area_code', 'Crop_Code']).apply(lambda x: calc_MAPE(x[nutrient], x[f'{nutrient}_F']))
    mape_per_nutrient = mape_per_nutrient.reset_index()
    mape_per_nutrient.columns = ['FAOStat_area_code', 'Crop_Code', f'MAPE_{nutrient}']
    mae_per_code = pd.merge(mae_per_code, mape_per_nutrient, on=['FAOStat_area_code', 'Crop_Code'])        
mae_per_code.replace(-10000000, pd.NA, inplace=True)
mae_per_code['MAE_NPK_avg_app'] = pd.NA
mae_per_code['MAPE_NPK_avg_app'] = pd.NA
for crop_code in df_merged_Pak['Crop_Code'].unique():
    subset = df_merged_Pak[df_merged_Pak['Crop_Code'] == crop_code]
    mae_per_code.loc[(mae_per_code['FAOStat_area_code'] == 165) & (mae_per_code['Crop_Code'] == crop_code),'MAE_NPK_avg_app'] = calc_MAE(subset['NPK_avg_app'],
                                                                                            subset['Total_avg_app'])
    mae_per_code.loc[(mae_per_code['FAOStat_area_code'] == 165) & (mae_per_code['Crop_Code'] == crop_code),'MAPE_NPK_avg_app'] = calc_MAPE(subset['NPK_avg_app'],
                                                                                            subset['Total_avg_app'])
df_c_codes = pd.read_excel('Countries_R_Codes.xlsx')
df_c_codes.drop(columns=['ISO3'],inplace=True)
mae_per_code = pd.merge(mae_per_code, df_c_codes, left_on=['FAOStat_area_code'], 
                      right_on=['FAOStat_area_code'])
mae_per_code.to_excel('test2.xlsx', index=False)
mae_per_code